# 1. 패키지 설치

In [ ]:
# poetry add pypdf=">=4.2.0,<5.0.0"
# poetry add langchain-upstage

# 2. 환경변수 불러오기

- `.env` 파일에 `UPSTAGE_API_KEY` 등록

In [ ]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[:4])

# 3. LLM 답변 생성

- Upstage Console에서 발급받은 API Key를 `UPSTAGE_API_KEY`라고 저장하면 별도의 설정 없이 `ChatUpstage`를 사용할 수 있음

In [ ]:
from langchain_upstage import ChatUpstage

#llm = ChatUpstage(temperature=0.5)
llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )

In [ ]:
ai_message=llm.invoke("LangChain은 무엇인가요?")
print(type(ai_message))
print(ai_message.content)

In [ ]:
# using chat stream
for chunk in llm.stream("LangChain은 무엇인가요?"):
    print(chunk.content, end="")

In [ ]:
from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate

translation_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a professional translator specializing in Korean-English translation."),
        ("human", "Translate this from {source_lang} to {target_lang}: {text}")
    ])

llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )

# 체인 실행
chain = translation_prompt | llm

response = chain.invoke({
    "source_lang": "English",
    "target_lang": "Korean", 
    "text": "LangChain is a powerful framework for building AI applications."
})

print("Upstage Response:")
print(response.content)
    

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )

# using chain
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that translates English to Korean."),
        ("human", "Translate this sentence from English to Korean. {english_text}."),
    ]
)

llm = ChatUpstage()
chain = prompt | llm

ai_message=chain.invoke({"english_text": "Hello, How are you?"})
print(ai_message.content)

#### Groundness Check
- Groundedness Check API는 사용자가 제공한 Context(컨텍스트)에 대한 AI 어시스턴트의 응답이 실제로 그 컨텍스트에 기반하고 있는지 여부를 확인합니다.

In [ ]:
from langchain_upstage import UpstageGroundednessCheck

groundedness_check = UpstageGroundednessCheck()

request_input = {
    "context": "삼성전자는 연결 기준으로 매출 74.07조원, 영업이익 10.44조원의 2024년 2분기 실적을 발표했다. 전사 매출은 전분기 대비 3% 증가한 74.07조원을 기록했다. DS부문은 메모리 업황 회복으로 전분기 대비 23% 증가하고, SDC는 OLED 판매 호조로 증가했다.",
    "answer": "삼성전자의 2024년 2분기 매출은 약 74.07조원이다.",
}

response = groundedness_check.invoke(request_input)
print(response)

#### UpstageEmbeddings

In [ ]:
from langchain_upstage import UpstageEmbeddings

embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
doc_result = embeddings.embed_documents(
    ["Sung is a professor.", "This is another document"]
)
print(doc_result)

query_result = embeddings.embed_query("What does Sung do?")
print(query_result)

#### UpdateDocumentLoader

In [ ]:
from langchain_upstage import UpstageDocumentParseLoader

file_path = "../data/tutorial-korean.pdf"
layzer = UpstageDocumentParseLoader(file_path, split="page")

# For improved memory efficiency, consider using the lazy_load method to load documents page by page.
docs = layzer.load()  # or layzer.lazy_load()
print(len(docs))

for doc in docs[:3]:
    print(doc)